In [1]:
# Scrape odds from odds portal
import time
import requests
import pandas as pd
import numpy as np
from datetime import date
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
# chrome_options.add_argument("--headless")
chrome_options.headless = False # also works
PATH = '/home/dev/Desktop/Projects/AI/chromedriver'
driver = webdriver.Chrome(PATH, options=chrome_options)

odds_df = ''
odds_list = []

pages = 31
headless = False

In [4]:
sport = 'basketball'
years = [i for i in range(2009, 2021)]
years

[2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

In [5]:
for year in years:
    
    headless = not headless
    chrome_options.headless = headless # also works
    driver = webdriver.Chrome(PATH, options=chrome_options)
    
    # Change timezone to Eastern USA
    driver.get('https://www.oddsportal.com/basketball/usa/nba/results/#/page/1/')
    time.sleep(3)
    timezone = driver.find_elements_by_class_name('user-header-fakeselect')
    timezone[1].click()
    time.sleep(2)
    driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[7]/div/a[40]').click()

    print("Scraping Year: ", year)
    # For loop through all pages
    for page in range(1, pages):
        print('Scraping page: ',page)

        driver.get(f'https://www.oddsportal.com/basketball/usa/nba-{year}-{year+1}/results/#/page/{page}/')

        time.sleep(3)

        table = driver.find_element_by_tag_name('tbody')

        rows = table.find_elements_by_tag_name('tr')

        for row in rows:
            try:
                date = row.find_element_by_css_selector('th.first2.tl').text
            except:
                cols = row.find_elements_by_tag_name('td')

                if len(cols) > 5:
                    teams = cols[1].text
                    score = cols[2].text
                    home_odds = cols[3].text
                    vis_odds = cols[4].text
                    odds_dict = {
                        'Date': date,
                        'Teams': teams,
                        'Score': score,
                        'Home Odds': home_odds,
                        'Vis Odds': vis_odds
                    }
                    odds_list.append(odds_dict)



    driver.quit()

    odds_df = pd.DataFrame(odds_list)
    odds_list.clear()


    # Manipulate the data

    # Split Teams into Home and Away
    odds_df[['Home','Visitor']] = odds_df['Teams'].str.split("-", expand=True,)
    odds_df['Home'] = odds_df['Home'].str.strip().replace('\n', '')   
    odds_df['Visitor'] = odds_df['Visitor'].str.strip()

    # Split Score into home and away
    odds_df['Score'] = odds_df['Score'].str.replace('OT', '')
    odds_df[['Home PTS', 'Vis PTS']] = odds_df.Score.str.split(':', expand=True)

    # Replace - with ''
    odds_df['Home Odds'].replace('-', '0', inplace=True)
    odds_df['Vis Odds'].replace('-', '0', inplace=True)

    # Drop Teams  and Score column column
    odds_df = odds_df.drop(['Teams', 'Score'], axis=1)

    # Replace strings from date
    odds_df['Date'] = odds_df['Date'].replace(regex=[' - All Stars', ' - Pre-season', ' - Play Offs', ' - Wild Card', ' - Promotion'], value = '')

    # Drop Nan values, ' - 
    odds_df = odds_df.dropna(subset=['Visitor', 'Vis PTS'])

    # Convert dates to datetime
    odds_df['Date'] = pd.to_datetime(odds_df['Date'])

    # Convert str to float
    odds_df[['Home Odds', 'Vis Odds', 'Home PTS', 'Vis PTS']] = odds_df[['Home Odds', 'Vis Odds', 'Home PTS', 'Vis PTS']].astype(float)

    # Create Home Points Diff and Home Win Column
    odds_df['Home Points Dif'] = odds_df['Home PTS'] - odds_df['Vis PTS']
    odds_df['Home Win'] = odds_df['Home PTS'] > odds_df['Vis PTS']

    # Convert date to datetime
    odds_df['Date'] = pd.to_datetime(odds_df['Date'], format='%Y-%m-%d')
    
    # Replace Team Names that changed
    home_vis_cols = ['Home', 'Visitor']
    for col in home_vis_cols:
        conditions = [
            (odds_df[col] == 'New Orleans Hornets'),
            (odds_df[col] == 'Charlotte Bobcats'),
            (odds_df[col] == 'New Jersey Nets'),
            True
        ]
        values = ['New Orleans Pelicans', 'Charlotte Hornets', 'Brooklyn Nets', odds_df[col]]
        odds_df[col] = np.select(conditions, values)
        
    # Save as CSV
    odds_df.to_csv(f'Games/NBA {year} Games.csv', index=False)

Scraping Year:  2009
Scraping page:  1
Scraping page:  2
Scraping page:  3
Scraping page:  4
Scraping page:  5
Scraping page:  6
Scraping page:  7
Scraping page:  8
Scraping page:  9
Scraping page:  10
Scraping page:  11
Scraping page:  12
Scraping page:  13
Scraping page:  14
Scraping page:  15
Scraping page:  16
Scraping page:  17
Scraping page:  18
Scraping page:  19
Scraping page:  20
Scraping page:  21
Scraping page:  22
Scraping page:  23
Scraping page:  24
Scraping page:  25
Scraping page:  26
Scraping page:  27
Scraping page:  28
Scraping page:  29
Scraping page:  30
Scraping Year:  2010
Scraping page:  1
Scraping page:  2
Scraping page:  3
Scraping page:  4
Scraping page:  5
Scraping page:  6
Scraping page:  7
Scraping page:  8
Scraping page:  9
Scraping page:  10
Scraping page:  11
Scraping page:  12
Scraping page:  13
Scraping page:  14
Scraping page:  15
Scraping page:  16
Scraping page:  17
Scraping page:  18
Scraping page:  19
Scraping page:  20
Scraping page:  21
Scrapin